## DV2599 Assignment 2
Group 8


***
Viktor Fransson

vifr22@student.bth.se

***

Tobias Gustafsson

togu22@student.bth.se
***

#### Initialize

In [29]:
# Import packages
import pandas as pd
import statistics as stat
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# Load dataset
df = pd.read_csv('spambase/spambase.data', delimiter=',', header=None)

# Add headers to columns to increase clarity and comprehensibility
headers = []
with open('spambase/spambase.names', 'r') as file:
    N = [line for line in file]
    headers = [x[:x.find(':')] for x in N[33:]]

headers.append("spam")  # Spam column is last, 1 = spam, 0 = not spam
df.columns = headers

# Split data into x and y
y = df['spam']
x = df.drop(columns='spam')

display(df)

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


#### 0. Pre-process

In [30]:
# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Initialize standard scaler
scaler = StandardScaler()

# Fit and transform train set
x_train_scaled = scaler.fit_transform(x_train)

# Transform test set
x_test_scaled = scaler.transform(x_test)

# Turn back into pandas dataframes
x_train_scaled = pd.DataFrame(x_train_scaled, columns=x_train.columns, index=x_train.index)
x_test_scaled = pd.DataFrame(x_test_scaled, columns=x_test.columns, index=x_test.index)

# Describe
x_train_scaled.describe().round(2)


,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,word_freq_conference,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total
count,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,...,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00,3680.00
mean,-0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00,...,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00
std,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
min,-0.35,-0.16,-0.55,-0.04,-0.46,-0.34,-0.28,-0.26,-0.32,-0.40,...,-0.11,-0.16,-0.59,-0.16,-0.32,-0.31,-0.10,-0.12,-0.38,-0.50
25%,-0.35,-0.16,-0.55,-0.04,-0.46,-0.34,-0.28,-0.26,-0.32,-0.40,...,-0.11,-0.16,-0.59,-0.16,-0.32,-0.31,-0.10,-0.11,-0.34,-0.44
50%,-0.35,-0.16,-0.55,-0.04,-0.46,-0.34,-0.28,-0.26,-0.32,-0.40,...,-0.11,-0.16,-0.31,-0.16,-0.32,-0.31,-0.10,-0.08,-0.27,-0.33
75%,-0.35,-0.16,0.26,-0.04,0.10,-0.34,-0.28,-0.26,-0.32,-0.13,...,-0.11,-0.16,0.22,-0.16,0.05,-0.11,-0.10,-0.04,-0.06,-0.03
max,13.26,11.56,9.32,32.17,14.03,20.63,17.76,26.95,17.98,18.48,...,32.18,17.16,21.92,38.20,37.33,24.41,43.34,33.50,17.08,16.24


#### 1. Stratified ten-fold cross-validation

In [35]:
# Initialize ten-fold cross-validation
skf = StratifiedKFold(n_splits=10)

# Initialize models
rf = RandomForestClassifier()
lr = LogisticRegression()
nb = GaussianNB()

models = [("Random Forest", rf), ("Logistic Regression", lr), ("Naive Bayes", nb)]
df_scores = pd.DataFrame({'Fold':[1,2,3,4,5,6,7,8,9,10,'avg', 'stdev']})

# Stratified K-fold for each model
for model_name, model in models:
    cv_score = cross_val_score(model, x_train_scaled, y_train, cv=skf, scoring="accuracy")
    df_scores[model_name] = np.append(cv_score, [stat.mean(cv_score), stat.stdev(cv_score)])

display(df_scores)

# Save to csv
df_scores.to_csv('df_scores_1.csv', index=False)

,Fold,Random Forest,Logistic Regression,Naive Bayes
0,1,0.932065,0.902174,0.817935
1,2,0.953804,0.910326,0.807065
2,3,0.956522,0.926630,0.831522
3,4,0.942935,0.921196,0.828804
4,5,0.951087,0.921196,0.809783
5,6,0.953804,0.937500,0.820652
6,7,0.967391,0.921196,0.809783
7,8,0.959239,0.951087,0.820652
8,9,0.964674,0.923913,0.790761
9,10,0.948370,0.907609,0.842391


#### 2. Results

#### 3. Friedman test

In [37]:
### Remove last two rows and fold column
df_friedman_score = df_scores.iloc[:-2]
df_rank = df_friedman_score.drop(columns='Fold')

### Rank
df_rank = df_rank.rank(1, 'max', ascending=False)

### Add avg_rank row
df_rank_avg = df_rank.mean()

### Combine ranks and score values
df_friedman_score = df_friedman_score.rename(columns={'Fold': 'Data set'})
for column in df_rank.columns:
    combines = [f"{round(x,4)} ({int(y)})" for x, y in zip(df_friedman_score[column], df_rank[column])]
    df_friedman_score[column] = combines

df_friedman_score.loc[len(df_friedman_score)] = ['avg rank', df_rank_avg.iloc[0], df_rank_avg.iloc[1], df_rank_avg.iloc[2]] 
df_friedman_score.to_csv('df_scores_2.csv', index=False)

display(df_friedman_score)

,Data set,Random Forest,Logistic Regression,Naive Bayes
0,1,0.9321 (1),0.9022 (2),0.8179 (3)
1,2,0.9538 (1),0.9103 (2),0.8071 (3)
2,3,0.9565 (1),0.9266 (2),0.8315 (3)
3,4,0.9429 (1),0.9212 (2),0.8288 (3)
4,5,0.9511 (1),0.9212 (2),0.8098 (3)
5,6,0.9538 (1),0.9375 (2),0.8207 (3)
6,7,0.9674 (1),0.9212 (2),0.8098 (3)
7,8,0.9592 (1),0.9511 (2),0.8207 (3)
8,9,0.9647 (1),0.9239 (2),0.7908 (3)
9,10,0.9484 (1),0.9076 (2),0.8424 (3)


#### 4. Significant average differences

##### Nemenyi test